## genAI RAG Demo

This notebook is based of [this tutorial](https://github.com/jedi4ever/learning-llms-and-genai-for-dev-sec-ops) repo from Patrick Debois (thanks Patrick!!)

- We'll be using the Langchain <https://github.com/langchain-ai/langchain> for our coding.
- We will be using Python but it also exists in a Typescript variant.

- These framework are a great way to track new ideas and examples through their documentation.

We recommend using [visual studio code](https://code.visualstudio.com/) for an editor and either running a local [virtual environment](https://code.visualstudio.com/docs/python/environments) or, if you're interested in [Docker](https://www.docker.com/), using a [dev container](https://code.visualstudio.com/docs/devcontainers/containers) (a devcontainer.json file is included with this repo).

This tutorial will walk through a basic RAG application.

### 1. Hello OpenAI: Installing dependencies for ["Hello, World!"](https://en.wikipedia.org/wiki/%22Hello,_World!%22_program)
If we're in a new virtual environment (see above), our first step install langchain and open AI dependencies with pip, the python package manager.

In [ ]:
%pip install langchain-openai langchain pypdf langchain-community chromadb

These frameworks allow the use of multiple different llms. Here we'll use OpenAI as an example.

Using OpenAI requires an API key `OPENAI_API_KEY` to be loaded. Python can use a `.env` file to store these values. Make sure you have either your personal or your hackathon team's OpenAI API key added to the .env file. VS code will automatically pick up the .env file, otherwise you may need to run the command below to install a package to load it.

In [ ]:
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

Now with the env loaded we make the simplest request by instantiating the OpenAI API and sending it a prompt.

In [ ]:
from langchain_openai import OpenAI
llm = OpenAI()
answer = llm.invoke("Hello world")
print(answer)

When we try this example a few times we'll see that the output is not consistent

In [ ]:
llm = OpenAI()
for i in range(1,6):
    answer = llm.invoke("Hello world")
    print(f"A{i}:{answer}")

Now to make the result more predictable we set the `temperature` option. This is the degree of randomness we allow the model to take in it's predictions

In [ ]:
llm = OpenAI(temperature=0)
for i in range(1,6):
    answer = llm.invoke("Hello world")
    print(f"A{i}:{answer}")

Some llms allow you to stream the results character by character. Feel free to change the prompt or the temperature and run the notebook cell below a few times.

In [ ]:
llm = OpenAI(streaming=True, temperature=0)
answer=llm.invoke("Where in the world is John Willis? try to answer verbose")
print(answer)

## 2. A simple OpenAI chat bot

The last example let us ask openAI a question in plain english. Now we'll explore setting up a simple chat bot.

Chat models typically operate using a set of messages they send to the LLM.
- The System message is what sets the tone or the initial behavior of the model
- The Human message that asks the question
- The AI Message that is the response of the AI

In [ ]:
# Initialize a chat model instead of a regular LLM
from langchain_openai import ChatOpenAI
chat = ChatOpenAI()

from langchain.schema import (
    SystemMessage,
    AIMessage,
    HumanMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant that provides security advice."),
    HumanMessage(content="What makes devOps secure?")
]
answer = chat.invoke(messages)
print(answer)

The next step is to create a template for the prompts we are sending.
Related Langchain lesson: <https://python.langchain.com/docs/concepts/prompt_templates/>


In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt = prompt_template.format(adjective="funny", content="AI")

from langchain_openai import OpenAI
llm = OpenAI(temperature=0)
llm.invoke(prompt)

Similar to a prompt template , we can also use chat prompt templates more optimized for conversations

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)

from langchain_openai.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0)
chat.invoke(messages)

## 3. Document Loading 

Now that we have the prompts loaded and the llm answering our questions, we dive into mixing the llm with our own content. This is useful because llms usually have a knowledge date cut off and also we don't want private information to leak into public llms. 

An important technique for this is called **RAG: Retrieval Augmented Generation**.

Langchain has a set of documentloader to load & parse different formats.

First, we need to install some more python dependencies. We'll be using **pypdf** here, but there a _lot_ of other options available on [langchain's website](https://python.langchain.com/docs/how_to/document_loader_pdf/).

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("PDFs/NIST.AI.100-1.pdf")
pages = loader.load_and_split()

# these are zero indexed, so "2" here is really page 3.
pages[2]

We can also load all the PDFs in the directory at once.

In [ ]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader(
    "./",
    glob="PDFs/**/*.*",
    use_multithreading=True,
    max_concurrency=4,
    show_progress=True,
    loader_cls=PyPDFLoader
)
pdf_pages = loader.load()
print('number of PDF pages:',len(pdf_pages))





## 4. Chunking & Splitting

Our PDF pages might be too long to fit in the context window of many models. Even for those models that could fit the full post in their context window, models can struggle to find information in very long inputs.

To handle this we’ll split the Document into chunks for embedding and vector storage. This should help us retrieve only the most relevant bits of the blog post at run time.

Chunking and splitting strategies can drastically improve the quality of results for a RAG application. This is a more advanced topic within genAI so we won't explore it further here, but your team may want to investigate how chunking and splitting could impact ther performance and quality of your use case.

In [ ]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(pdf_pages)


all_splits[10]


## 5. Embeddings & Vector stores

Now that we have the PDFs loaded, parsed, chunked, and split they're starting to sound like and order of waffle house hashbrowns. The next step is to calculate an "embedding" of the texts. 

An embedding in its simplest form is a multi dimensional mathematical vector that calculates the similarity of a piece of text. Topics and content that are similar have vectors that are close together. 

Embeddings enhance traditional search engines that use keywords or synonyms. Both have their place, but in this example we'll use embeddings as a way to calculate the proximity of texts.

Why similarity you might ask? Well, RAG allows us to provide an LLM with more context about a subject when we ask it a question. This context is done by looking up documents that have similarity and giving that information. So in this section we're laying the groundwork for that end-goal.

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
print(f"Embeddings calculated: {len(embeddings)}")
print(f"Vectorsize of first embedding :{len(embeddings[0])}")



Vectors are just like vectors found in some math or physics courses, so they indicate a magnitude (number), and a direction(+/- at a minimum). Most real world vectors have about 3 dimensions. These vectors have a lot more. 

In [ ]:
print(embeddings[2])

Instead of calculating these embeddings everytime, we'll store these vectors into a **vector database** or a cache. Caching is another advanced topic outside the scope of this quickstart that your team may want to investigate further.

In many cases, vector databases also provide the service to calculate the embeddings using the model you select. For this example we'll be using [Chroma DB](https://www.trychroma.com/).

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
# Set the embeddings function
embeddings_model = OpenAIEmbeddings()

# Vectory database will calculate them using the embeddings_model provided
# and store the embeddings for each doc in it's database
db = Chroma.from_documents(all_splits, embeddings_model)

In [ ]:
from pprint import pprint
query = "devOps"
docs = db.similarity_search_with_relevance_scores(query, k=4, score_threshold=0.7)
pprint(docs)

Scrolling all the way to the right to check the metadata, we can see The code returned a few splits that point to either the table of contents or page 4 (3+1) of the Great_Dane_Maintenance_Manual where the the freezing weather maintenance is discussed.

## 6 Question Answering Retrival Augmented Generation (QA RAG)

We're almost there!
- We loaded the documents.
- We've split them in to relevant parts (splitters)
- We've had the vector database calculate the embeddings and store the related parts

Related Langchain example: https://python.langchain.com/docs/use_cases/question_answering/

Now We'll set up our QA chain with our LLM and our DB as a retriever. 

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What makes devOps secure?")

## Conclusion & Next Steps

- Can our QA RAG bot answer questions more accurately than vectore simularity search alone?
- How might the data included or excluded from the vectorstore along with how it's split and chunked affect the results?

Thanks, and happy hacking!
